<a href="https://colab.research.google.com/github/Necl0/Bonsai/blob/main/kmnistCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import torch
import torch.nn as nn
import torchvision
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Hyper Parameters
num_epochs = 5
num_classes = 10
batch_size = 32
learning_rate = 0.001


# load KMNIST
train_data = torchvision.datasets.KMNIST(root='../../data/',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)

test_data = torchvision.datasets.KMNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data Loader
train_loader = DataLoader(dataset=train_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = DataLoader(dataset=test_data,
                                          batch_size=batch_size, 
                                          shuffle=False)



  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ../../data/KMNIST/raw/train-images-idx3-ubyte.gz to ../../data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ../../data/KMNIST/raw/train-labels-idx1-ubyte.gz to ../../data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ../../data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ../../data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/KMNIST/raw



In [4]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')


Epoch [1/5], Step [100/1875], Loss: 0.3330100476741791
Epoch [1/5], Step [200/1875], Loss: 0.4970211982727051
Epoch [1/5], Step [300/1875], Loss: 0.09238164871931076
Epoch [1/5], Step [400/1875], Loss: 0.4852636754512787
Epoch [1/5], Step [500/1875], Loss: 0.18198931217193604
Epoch [1/5], Step [600/1875], Loss: 0.1939224749803543
Epoch [1/5], Step [700/1875], Loss: 0.0753738060593605
Epoch [1/5], Step [800/1875], Loss: 0.05381755530834198
Epoch [1/5], Step [900/1875], Loss: 0.10258735716342926
Epoch [1/5], Step [1000/1875], Loss: 0.09786395728588104
Epoch [1/5], Step [1100/1875], Loss: 0.11136063188314438
Epoch [1/5], Step [1200/1875], Loss: 0.06502556055784225
Epoch [1/5], Step [1300/1875], Loss: 0.05651417747139931
Epoch [1/5], Step [1400/1875], Loss: 0.046838682144880295
Epoch [1/5], Step [1500/1875], Loss: 0.2164844274520874
Epoch [1/5], Step [1600/1875], Loss: 0.10267018526792526
Epoch [1/5], Step [1700/1875], Loss: 0.22033579647541046
Epoch [1/5], Step [1800/1875], Loss: 0.052160

In [8]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy of the model on the 10000 test images: {100 * correct / total}')

Test Accuracy of the model on the 10000 test images: 93.77


In [9]:
import pickle

with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [10]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 7.6 MB/s 
     |████████████████████████████████| 4.7 MB 41.4 MB/s 
     |████████████████████████████████| 78 kB 9.3 MB/s 
     |████████████████████████████████| 235 kB 74.1 MB/s 
     |████████████████████████████████| 181 kB 62.8 MB/s 
     |████████████████████████████████| 164 kB 57.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 51 kB 9.0 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=c616dd887cead144d82798f99309ccaae6401219233699b5277925f20ffd5c6e
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators
